In [1]:
import pandas as pd
import numpy as np
import re
from time import time
import sys
import pickle
import os

from keras.engine.topology import Layer
from keras.layers import InputSpec
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D

from keras import backend as K
from keras.utils import conv_utils, to_categorical
from keras.datasets import mnist

from stochastic_pooling import *
from mixed_pooling import *

import math
import random
random.seed(91)

/home/syht/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)[:1000,:,:,:]
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)[:100,:,:,:]
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train[:1000], num_classes)
y_test = to_categorical(y_test[:100], num_classes)

x_train shape: (1000, 28, 28, 1)
1000 train samples
100 test samples


In [6]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    #model.add(StochasticPooling(pool_size=(2, 2), modify=False))
    model.add(MixedPooling(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    #model.add(StochasticPooling(pool_size=(2, 2), modify=False))
    model.add(MixedPooling(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()
    return model

In [7]:
model = get_model()

Train old alpha: [0. 0.]
Train new alpha: [0.16135967 0.83864033]
Test phase: [0.16135967 0.83864033]
Train old alpha: [0. 0.]
Train new alpha: [0.84432802 0.15567198]
Test phase: [0.84432802 0.15567198]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
mixed_pooling_2 (MixedPoolin (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
mixed_pooling_3 (MixedPoolin (None, 6, 6, 16)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 6, 16)          0         
_________________________________________________________________
flat

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 1000 samples, validate on 100 samples
Epoch 1/20
1000/1000 [==============================] - 2s 2ms/step - loss: 2.1287 - acc: 0.2620 - val_loss: 1.5586 - val_acc: 0.6000
Epoch 2/20
1000/1000 [==============================] - 1s 1ms/step - loss: 1.3764 - acc: 0.5380 - val_loss: 0.7004 - val_acc: 0.7800
Epoch 3/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.8830 - acc: 0.7010 - val_loss: 0.5150 - val_acc: 0.8600
Epoch 4/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6882 - acc: 0.7780 - val_loss: 0.4216 - val_acc: 0.8400
Epoch 5/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5929 - acc: 0.8140 - val_loss: 0.3621 - val_acc: 0.8600
Epoch 6/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5101 - acc: 0.8300 - val_loss: 0.2449 - val_acc: 0.9400
Epoch 7/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4468 - acc: 0.8670 - val_loss: 0.2028 - val_acc: 0.9600
Epoch 8/20
1000

In [6]:
def custom_generator(X, y, batch_size=32):
    while 1:
        for i in range(math.ceil(len(X)/batch_size)):
            X_batch = X[batch_size*i : batch_size*(i+1)]
            y_batch = y[batch_size*i : batch_size*(i+1)]
            
            yield(X_batch, y_batch)

In [7]:
gen_train = custom_generator(x_train, y_train, batch_size)
gen_test = custom_generator(x_test, y_test, batch_size)

In [8]:
model.fit_generator(gen_train, 
                        steps_per_epoch=math.ceil(len(x_train)/batch_size), 
                        epochs=epochs,
                        verbose=1,
                        validation_data=gen_test, 
                        validation_steps=math.ceil(len(x_test)/batch_size))

Epoch 1/1
32/32 [==============================] - 278s 9s/step - loss: 1.4846 - acc: 0.5290 - val_loss: 0.8102 - val_acc: 0.7500
